# Clustering Crypto

In [22]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [23]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
print(crypto_df.shape)
crypto_df.head()

(1252, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [24]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [25]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(['IsTrading'], inplace=False, axis=1)
print(crypto_df.shape)
crypto_df.head()

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [26]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
print(crypto_df.shape)
crypto_df.head()

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [27]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
print(crypto_df.shape)
crypto_df.head()

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [28]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = crypto_df.filter(['CoinName'], axis=1)
crypto_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [29]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop('CoinName', inplace=False, axis=1)
print(crypto_df.shape)
crypto_df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [30]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
print(X.shape)
X.head()

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [32]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
# Get three principal components from the data
X_pca = pca.fit_transform(X_scaled)

In [33]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data = X_pca, columns=["PC 1", "PC 2", "PC 3"]
)
# Set the index of a df1 to a df2
pcs_df.index=X.index
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.330778,0.873532,-0.609309
404,-0.314117,0.873786,-0.609704
1337,2.295752,1.705135,-0.697516
BTC,-0.145316,-1.236852,0.177111
ETH,-0.156435,-1.992456,0.396906


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [34]:
# Create an elbow curve to find the best value for K.
inertia = []
    
k = list(range(1, 11))

# Calculate the inertia for the range of K values

for i in k:
    
    km = KMeans(n_clusters=i, random_state=0)
    
    km.fit(pcs_df)
    
    inertia.append(km.inertia_)
    
# Create the elbow curve

elbow_data = {"k": k, "inertia": inertia}

df_elbow = pd.DataFrame(elbow_data)

df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [35]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([3, 3, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3,
       0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0,
       0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 3, 0, 0, 0, 0, 3,
       3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0,
       3, 0, 3, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3,
       0, 3, 0, 0, 3, 0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 3, 0, 3, 3,
       3, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3, 3, 3, 3,
       0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3,
       3, 0, 3, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 3, 0,
       3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 0, 3,

In [36]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
## pcs_df['class'] = model.labels_
clustered_df = crypto_df.join(pcs_df, how='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_names_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.330778,0.873532,-0.609309,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.314117,0.873786,-0.609704,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.295752,1.705135,-0.697516,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145316,-1.236852,0.177111,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.156435,-1.992456,0.396906,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.160270,-1.264149,-0.041900,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.406823,1.385061,-0.454454,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.146458,-2.208258,0.449259,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.154877,-1.992547,0.396888,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.169361,-1.886077,0.391752,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [40]:
# Creating a 3D-Scatter with the PCA data and the clusters
# three_clusters = get_clusters(3, pcs_df)
# three_clusters.head()
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"]
)

fig.update_layout(legend=dict(x=0, y=1))

fig.show()

In [42]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm","ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"])



:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [48]:
# Print the total number of tradable cryptocurrencies.
n = clustered_df.shape[0]
print(f"There are {n} tradable cryptocurrencies")

There are 532 tradable cryptocurrencies


In [49]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler().fit_transform(X)
print(scaler)

[[0.00000000e+00 4.20000000e-11 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.06585544e-03 5.32000000e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.95755135e-02 3.14159265e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.90135079e-04 1.40022261e-03 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.37028150e-06 2.10000000e-05 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.29582282e-07 1.00000000e-06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [66]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    data = scaler[:,[0, 1]]
)

plot_df.index=X.index

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = crypto_names_df


# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = model.labels_

plot_df.head(10)

,0,1,CoinName,Class
42,0.000000,4.200000e-11,42 Coin,3
404,0.001066,5.320000e-04,404Coin,3
1337,0.029576,3.141593e-01,EliteCoin,3
BTC,0.000018,2.100000e-05,Bitcoin,0
ETH,0.000109,0.000000e+00,Ethereum,0
LTC,0.000064,8.400000e-05,Litecoin,0
DASH,0.000009,2.200000e-05,Dash,3
XMR,0.000017,0.000000e+00,Monero,0
ETC,0.000115,2.100000e-04,Ethereum Classic,0
ZEC,0.000007,2.100000e-05,ZCash,0


In [70]:
plot_df.columns = ['TotalCoinsMined', 'TotalCoinSupply', 'CoinName', 'Class']
plot_df = plot_df[['TotalCoinSupply', 'TotalCoinsMined', 'CoinName', 'Class']]

plot_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,3
404,5.320000e-04,0.001066,404Coin,3
1337,3.141593e-01,0.029576,EliteCoin,3
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0


In [74]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x = "TotalCoinsMined",
    y = "TotalCoinSupply",
    hover_cols = "CoinName",
    by = "Class"
    
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)